In [ ]:
from google.colab import files
uploaded=files.upload()


Saving cleaned_bbc_news_articles (1).csv to cleaned_bbc_news_articles (1).csv


In [ ]:
import pandas as pd

# Replace 'your_dataset.csv' with the path to your dataset
df = pd.read_csv('cleaned_bbc_news_articles (1).csv')


In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Define text cleaning function
def clean_text(text):
    text = text.lower()
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply cleaning
df['cleaned_text'] = df['title'].apply(clean_text)


<ipython-input-14-85b0160d2e97>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

df['pubDate'] = pd.to_datetime(df['pubDate'])
df['month'] = df['pubDate'].dt.to_period('M')
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])
feature_names = tfidf_vectorizer.get_feature_names_out()

# Function to get top n terms and filter out unwanted terms
def get_top_n_terms(tfidf_scores, feature_names, n=10, unwanted_terms=None):
    # Ensure tfidf_scores is a 1D array
    tfidf_scores = np.array(tfidf_scores).flatten()

    # Sort scores and get top n indices
    sorted_indices = np.argsort(tfidf_scores)[::-1][:n]
    top_n_terms = [feature_names[i] for i in sorted_indices]

    # Filter out unwanted terms
    if unwanted_terms is not None:
        top_n_terms = [term for term in top_n_terms if term not in unwanted_terms]

    return top_n_terms

# Define unwanted terms
unwanted_terms = {'aa'}

# Group by month and extract top 10 terms
top_terms_per_month = {}
for month in df['month'].unique():
    month_df = df[df['month'] == month]
    month_tfidf_matrix = tfidf_vectorizer.transform(month_df['cleaned_text'])
    mean_tfidf_scores = month_tfidf_matrix.mean(axis=0).flatten().tolist()  # Convert to list
    top_terms = get_top_n_terms(mean_tfidf_scores, feature_names, n=10, unwanted_terms=unwanted_terms)
    top_terms_per_month[month] = top_terms

# Display the top terms for each month
for month, terms in top_terms_per_month.items():
    print(f"Top terms for {month}:")
    print(', '.join(terms))  # Convert list to comma-separated string
    print()

Top terms for 2022-03:
ukraine, war, russian, russia, uk, covid, refugee, ukrainian, putin, say

Top terms for 2022-02:
covid, test, still, need, selfisolation, protected, england, vulnerable, vaccination, abroad

Top terms for 2022-01:
look, keeping, covid, home, promise, conservative, election, flagship, flag, flagged

Top terms for 2022-04:
ukraine, war, russian, russia, election, uk, covid, world, say, ukrainian

Top terms for 2021-12:
headingley, inning, extraordinary, stokes, ben, ash, australia, england, flaming, flamingo

Top terms for 2022-05:
ukraine, war, election, russian, say, uk, final, russia, cost, ukrainian

Top terms for 2022-06:
ukraine, jubilee, war, england, platinum, new, strike, rail, queen, say

Top terms for 2019-10:
grenfell, tower, happened, zwerner, flake, flagged, flagging, flagpole, flagrant, flagship

Top terms for 2021-03:
cladding, grenfell, tower, thing, inquiry, know, flair, flack, flag, flagged

Top terms for 2022-07:
euro, england, wimbledon, uk, le